In [ ]:
import numpy as np
from toolkit.tire_model.tire_model_pacejka_2010 import *
from toolkit.tire_model.tire_model_fitting import *
from toolkit.tire_model.ttc_loader import *
from toolkit.tire_model.tire_fitting_masks import *
from toolkit.tire_model.tire_model_utils import *

In [ ]:
# Get the ttc data for the tire you want to fit
combi_runs, cornering, drive_brake, name = load_runs(get_R20_18x6_7_runs(), smooth=True)

In [ ]:
# If you want to start from the default tir file
tire_model = readTIR(make_path('./Data/TTCData/TIR_Templates/FSAE_Defaults.tir'))
# or from a existing set of parameters
# tire_model = tire_model_from_arr(H_R20_18X6_6)

In [ ]:
tire_model.UNLOADED_RADIUS = 0.223
tire_model.NOMPRES = 83000
tire_model.FNOMIN = 1100 # the max normal force if the model is limited to 2 times this value
tire_model.PKX1 = 48
tire_model.PKX2 = 15 # the default causes you to need a very high fnomin
tire_model.PKX3 = -0.3

We should look into LMA as a alternative to the GNA that we are using now in the least squares fitting (see https://en.wikipedia.org/wiki/Levenberg%E2%80%93Marquardt_algorithm)

In [ ]:
# filter the data to get only IA = 0, PHIT ~ 0, and SA not in a transient state (when it inverts)
df = remove_time_gaps(filter_ia(filter_vel(filter_press(cornering, press=82700, d_press=6000), 11.1, 0.1)))
df = df[np.abs(df.SL) == 0]
df = df[df.TSTC > 50]
run_fit(tire_model, df, fit_f_y, FY_MASK_IND)

In [ ]:
# filter the data to get only IA = 0, PHIT ~ 0, and SA not in a transient state (when it inverts)
df = remove_time_gaps(filter_ia(filter_vel(cornering, 11.1, 0.1)))
df = df[np.abs(df.SL) == 0]
df = df[df.TSTC > 50]
run_fit(tire_model, df, fit_f_y, FY_MASK_PRESS)

In [ ]:
# filter the data to get only IA = 0, PHIT ~ 0, and SA not in a transient state (when it inverts)
df = remove_time_gaps(filter_vel(cornering, 11.1, 0.1))
df = df[np.abs(df.SL) == 0]
df = df[df.TSTC > 50]
run_fit(tire_model, df, fit_f_y, np.concatenate((FY_MASK_IA, FY_MASK_IA_PRESS)), graph=True)

In [ ]:
# Fit Mz to combined cornering and drive brake data
df = remove_time_gaps(filter_ia(filter_vel(filter_press(cornering, press=82700, d_press=6000), 11.1, 0.1)))
df = df[np.abs(df.SL) == 0]
df = df[df.TSTC > 50]
run_fit(tire_model, df, fit_m_z, MZ_MASK_IND)

In [ ]:
# Fit Mz to combined cornering and drive brake data
df = remove_time_gaps(filter_ia(filter_vel(cornering, 11.1, 0.1)))
df = df[np.abs(df.SL) == 0]
df = df[df.TSTC > 50]
run_fit(tire_model, df, fit_m_z, MZ_MASK_PRESS)

In [ ]:
# Fit Mz to combined cornering and drive brake data
df = remove_time_gaps(filter_vel(cornering, 11.1, 0.1))
df = df[np.abs(df.SL) == 0]
df = df[df.TSTC > 50]
run_fit(tire_model, df, fit_m_z, MZ_MASK_IA, graph=True)

In [ ]:
# filter the data to get only IA = 0, PHIT ~ 0, and SA not in a transient state (when it inverts)
df = remove_time_gaps(filter_vel(drive_brake, 11.1, 0.1))
df = df[np.abs(df.SL) < 0.01]
df = df[df.TSTC > 50]
run_fit(tire_model, df, fit_f_y, FY_LAMBDA, graph=True, long=True)

In [ ]:
# filter the data to get only IA = 0, PHIT ~ 0, and SA not in a transient state (when it inverts)
df = remove_time_gaps(filter_vel(drive_brake, 11.1, 0.1))
df = df[np.abs(df.SL) != 0]
df = df[df.TSTC > 50]
# Fit Fy to combined cornering and drive brake data
fy_mask_com_new = FY_MASK_COM.copy()
fy_mask_com_new.remove(NAMES.index("RBY3"))
fy_mask_com_new.remove(NAMES.index("REY1"))
fy_mask_com_new.remove(NAMES.index("REY2"))
run_fit(tire_model, df, fit_f_y, fy_mask_com_new, graph=True, long=True)

In [ ]:
# Fit Mz to combined cornering and drive brake data
df = remove_time_gaps(filter_vel(combi_runs, 11.1, 0.1))
df = df[np.abs(df.SL) != 0]
df = df[df.TSTC > 50]
run_fit(tire_model, df, fit_m_z, MZ_MASK_COM, graph=True, long=True)

In [ ]:
params = tire_model.dump_params()
params[FY_LAMBDA] = 1.0
set_x(params, tire_model)

In [ ]:
# Fit Fx to combined cornering and drive brake data
df = remove_time_gaps(filter_ia(filter_vel(filter_press(drive_brake, press=82700, d_press=6000), 11.1, 0.1)))
df = df[np.abs(df.SL) != 0]
df = df[np.abs(df.SA) < np.deg2rad(1)]
df = df[df.TSTC > 50]
fx_mask_ind_new = FX_MASK_IND.copy()
# fx_mask_ind_new.remove(NAMES.index("PKX1"))
run_fit(tire_model, df, fit_f_x, fx_mask_ind_new)

In [ ]:
# Fit Fx to combined cornering and drive brake data
df = remove_time_gaps(filter_ia(filter_vel(drive_brake, 11.1, 0.1)))
df = df[np.abs(df.SL) != 0] # the cornering runs arent useful here
df = df[np.abs(df.SA) < np.deg2rad(1)]
df = df[df.TSTC > 50]
run_fit(tire_model, df, fit_f_x, FX_MASK_PRESS)

In [ ]:
# Fit Fx to combined cornering and drive brake data
df = remove_time_gaps(filter_vel(drive_brake, 11.1, 0.1))
df = df[np.abs(df.SL) != 0] # the cornering runs arent useful here
df = df[np.abs(df.SA) < np.deg2rad(1)]
df = df[df.TSTC > 50]
run_fit(tire_model, df, fit_f_x, FX_MASK_IA, graph=True, long=True)

In [ ]:
# Fit Fx to combined cornering and drive brake data
df = remove_time_gaps(filter_vel(combi_runs, 11.1, 0.1))
df = df[df.TSTC > 50]
df = df[np.abs(df.SA) > np.deg2rad(1)]
fx_mask_com_new = FX_MASK_COM.copy()
fx_mask_com_new.remove(NAMES.index("RHX1"))
run_fit(tire_model, df, fit_f_x, fx_mask_com_new, graph=True)

In [ ]:
out = parameter_estimation_function(tire_model.dump_params()[MX_MASK_IND_PRESS], remove_time_gaps(filter_vel(combi_runs)), tire_model, params_list, error_list, fit_f_y, MX_MASK_IND_PRESS)
rms_error_fy = np.sqrt(np.mean(np.square(out)))
out = parameter_estimation_function(tire_model.dump_params()[MX_MASK_IND_PRESS], remove_time_gaps(filter_vel(combi_runs)), tire_model, params_list, error_list, fit_f_x, MX_MASK_IND_PRESS)
rms_error_fx = np.sqrt(np.mean(np.square(out)))
out = parameter_estimation_function(tire_model.dump_params()[MX_MASK_IND_PRESS], remove_time_gaps(filter_vel(combi_runs)), tire_model, params_list, error_list, fit_m_z, MX_MASK_IND_PRESS)
rms_error_mz = np.sqrt(np.mean(np.square(out)))
print(f"RMS Errors: Fy:{rms_error_fy:.2f}, Fx:{rms_error_fx:.2f}, Mz:{rms_error_mz:.2f}")
tire_model_two = tire_model
tire_model_two.PVY1 = 0
tire_model_two.PVY2 = 0
tire_model_two.PHY1 = 0
tire_model_two.PHY2 = 0
out = parameter_estimation_function(tire_model_two.dump_params()[MX_MASK_IND_PRESS], remove_time_gaps(filter_vel(combi_runs)), tire_model_two, params_list, error_list, fit_f_y, MX_MASK_IND_PRESS)
rms_error_fy = np.sqrt(np.mean(np.square(out)))
out = parameter_estimation_function(tire_model_two.dump_params()[MX_MASK_IND_PRESS], remove_time_gaps(filter_vel(combi_runs)), tire_model_two, params_list, error_list, fit_f_x, MX_MASK_IND_PRESS)
rms_error_fx = np.sqrt(np.mean(np.square(out)))
out = parameter_estimation_function(tire_model_two.dump_params()[MX_MASK_IND_PRESS], remove_time_gaps(filter_vel(combi_runs)), tire_model_two, params_list, error_list, fit_m_z, MX_MASK_IND_PRESS)
rms_error_mz = np.sqrt(np.mean(np.square(out)))
print(f"RMS Errors: Fy:{rms_error_fy:.2f}, Fx:{rms_error_fx:.2f}, Mz:{rms_error_mz:.2f}")

In [ ]:
# Generate graphs to check the fit
# split_run_with_MF_SR(drive_brake, tire_model, f"After Fy, Fx, Mz, and Mx Optimization")
split_run_with_MF_SR(combi_runs, tire_model, f"After Fy, Fx, Mz, and Mx Optimization")
# split_run_fit(drive_brake, tire_model, f"After Fy Optimization")
# split_run_with_MF_SA(cornering, tire_model, f"After Fy, Fx, Mz, and Mx Optimization")
# split_run_fit(cornering, tire_model, f"After Fy Optimization")
# split_run_fit(drive_brake, tire_model, f"After Fy Optimization")


In [ ]:
print(tire_model.PKX1)
print(tire_model.PKX2)
print(tire_model.PKX3)

In [ ]:
# Generate graphs to check smoothness of the fit
sweep_SA(tire_model, "Slip Angle")
sweep_SR(tire_model, "Slip Ratio")

In [ ]:
# Dump the parameters to a list
# dump_param(tire_model.dump_params())
dump_param(tire_model_two.dump_params())


In [ ]:
# Write out the parameters with their names and descriptions
tire_model_2 = tire_model_from_arr(H_R25B_18X6_6)#readTIR(make_path('./Data/TTCData/TIR_Templates/FSAE_Defaults.tir'))
param = tire_model_2.dump_params()
params = tire_model.dump_params()
for i in FX_MASK_IND:
        print(f"{i}\t{params[i]:.4f}\t{LABELS[i]}\t{NAMES[i]}\tDefault:{param[i]:.4f}")

In [ ]:
# Write out the parameters to a file
writeTIR(make_path('./Data/TTCData/TIR_Templates/H_R20_18X6_7.tir'), tire_model)